In [12]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [13]:
llm_config = {
    "model": "gpt-4-turbo",
    "api_key": OPENAI_API_KEY,
}

In [14]:
from autogen.coding import LocalCommandLineCodeExecutor

executor = LocalCommandLineCodeExecutor(
    timeout=60,
    work_dir=".",
)

In [15]:
from autogen import ConversableAgent, AssistantAgent

In [16]:
code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="ALWAYS",
    default_auto_reply="Please continue. If everything is done, reply TERMINATE",
)

In [17]:
code_writer_agent = AssistantAgent(
    name="code_writer_agent",
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [18]:
import datetime

today = datetime.datetime.now().date()

message = f"Today is {today}. Create a plot showing gain YDT for NVDA and TSLA. Make sure the code is in markdown code block and save the figure to a file ydt_stock_gains.png"

In [20]:
chat_result = code_executor_agent.initiate_chat(code_writer_agent, message=message)

code_executor_agent (to code_writer_agent):

Today is 2025-01-27. Create a plot showing gain YDT for NVDA and TSLA. Make sure the code is in markdown code block and save the figure to a file ydt_stock_gains.png

--------------------------------------------------------------------------------
code_writer_agent (to code_executor_agent):

To accomplish this task, we will follow these steps:

1. **Fetch Stock Data**: Get historical stock data for NVDA and TSLA for the year 2025.
2. **Compute Gain YTD**: Calculate the Year-to-Date (YTD) gains for both stocks starting from the first trading day of 2025.
3. **Plot the Data**: Generate a plot showing the YTD gain for both stocks.
4. **Save the Plot**: Save the plot to a file named `ydt_stock_gains.png`.

We can use the `yfinance` library to fetch the stock data, `pandas` for data manipulation, and `matplotlib` for plotting. Here's the complete Python code:

```python
# filename: plot_stock_gains.py
import yfinance as yf
import matplotlib.pyplo